In [1]:
from em_examples.LinearInversion import *
from ipywidgets import interact, FloatSlider, ToggleButtons, IntSlider, FloatText, IntText

In [2]:
app = LinearInversionApp()

# Linear Inversion App

This app is based upon the inversion tutorial: "INVERSION FOR APPLIED GEOPHYSICS" by Oldenburg and Li (2005).

Douglas W. Oldenburg and Yaoguo Li (2005) 5. Inversion for Applied Geophysics: A Tutorial. Near-Surface Geophysics: pp. 89-150.
eISBN: 978-1-56080-171-9 
print ISBN: 978-1-56080-130-6 
https://doi.org/10.1190/1.9781560801719.ch5 


## Purpose
By using a simple decaying and oscillating kernel function, which emulates physics of electromagnetic (EM) survey, we understand basic concepts of inverting EM data. Three main action items that we are going to explore are:

- Step1: Generate a sensitivity kernel (or matrix), $\mathbf{G}$
- Step2: Set a model then Simulate data ($\mathbf{d} = \mathbf{G}\mathbf{m}$)
- Step3: Invert the data, and explore inversion results

## Forward problem

- why is j in exponential
- plot g (so amplitude will be unity at x=0) (not rows of G)
- why jk k=1..N;   this can be j  (jmin, jmax): N is the number of data ( N-1 increments between jmin and jmax)
- k1=jmin; k2=jmax
- allow for a negative p:  enter in a box
- Rows of G:   g_1, g_N
- vmin -->  ymin
- scale on singular values 

In order to emulate EM data, which are often decaying and oscillating, we consider a kernel, $g_j(x)$, for $j$th datum. With a given model $m(x)$, $j$th datum can be computed by solving following integral equation:

 $$ d_j = \int_a^{b} g_j(x) m(x) dx $$

where 

$$ g_j(x) = e^{jpx} cos (2 \pi jqx) $$ 

By discretizing $g_j(x)$ we obtain

$$ \mathbf{g}_j(\mathbf{x}) = e^{jp\mathbf{x}} cos (2 \pi jq \mathbf{x})$$

where

- $\mathbf{g}_j$: $j$th row vector for the sensitivty matrix ($1 \times M$)
- $\mathbf{x}$: model location ($1 \times M$)
- $j_k$: k=1, 2, ..., N
- $p$: decaying constant (<0)
- $q$: oscillating constant (>0)

By stacking multiple rows of $\mathbf{g}_j$, we obtain sensitivity matrix, $\mathbf{G}$: 

\begin{align}
    \mathbf{G} = 
    \begin{bmatrix}
        \mathbf{g}_1\\
        \vdots\\
        \mathbf{g}_{N}
    \end{bmatrix}
\end{align}

Here, the size of the matrix $\mathbf{G}$ is $(N \times M)$. 
Finally data, $\mathbf{d}$, can be written as a linear equation:

$$ \mathbf{d} = \mathbf{G}\mathbf{m}$$

where $\mathbf{m}$ is an inversion model; this is a column vector ($M \times 1$). 

In real measurments, there will be various noises source, and hence observation, $\mathbf{d}^{obs}$, can be written as 

$$ \mathbf{d}^{obs} = \mathbf{G}\mathbf{m} + \mathbf{noise}$$

##  Step1: Generate a sensitivity kernel (or matrix), $\mathbf{G}$

By using the following app, we explore each row vector of the sensitivity matrix, $\mathbf{g}_j$. Parameters of the apps are:

- M: # of model parameters
- N: # of data
- p: decaying constant (<0)
- q: oscillating constant (>0)
- ymin: maximum limit for y-axis
- ymax: minimum limit for y-axis

In [3]:
Q1 = app.interact_plot_G()

interactive(children=(IntSlider(value=20, continuous_update=False, description='N', min=1), IntSlider(value=100, continuous_update=False, description='M', min=1), FloatSlider(value=-0.25, continuous_update=False, description='p', max=0.0, min=-1.0, step=0.05), FloatSlider(value=0.25, continuous_update=False, description='q', max=1.0, step=0.05), FloatText(value=1.0, description='k1'), FloatText(value=19.0, description='kn'), FloatText(value=-0.005, description='ymin'), FloatText(value=0.011, description='ymax'), ToggleButtons(description='scale', index=1, options=('linear', 'log'), value='log'), Output()), _dom_classes=('widget-interact',))

## Step2: Generate a model and simulate data



The model $m$ is a function defined on the interval (0,1). Here we generate a model that is the sum of a: (a) background $m_{ref}$, (b) box car $m_1$ and (c) Gaussian $m_2$. The box car is defined by
- m1 : amplitude
- $m1_{center}$ : center
- $m1_{width}$ : width
the Gaussian is defined by 
- m2 : amplitude
- $m2_{center}$ : center
- $m2_{sigma}$ : width of Gaussian (as defined by a standard deviation)

The $j$th datum is the inner product of the $jth$ kernel $g_j(x)$ and the model $m(x)$. In discrete form it can be written as the dot product of the vector $g_j$ and the model vector $m$.

### $$ d_j = \mathbf{g}_j \mathbf{m} $$

If there are $N$ number of data, these data can be written as a column vector, $\mathbf{d}$:

\begin{align}
    \mathbf{d} = \mathbf{G}\mathbf{m} = 
    \begin{bmatrix}
        d_1\\
        \vdots\\
        d_{N}
    \end{bmatrix}
\end{align}

### Adding Noise

Observational data are always contaminated with noise. Here we add Gaussian noise $N(0,\epsilon)$ (zero mean and standard deviation $\sigma$). Here we choose 

$$ \epsilon = \% |d| + \text{floor} $$
  

In [6]:
Q2 = app.interact_plot_model()

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='m_background', max=2.0, min=-2.0, step=0.05), FloatSlider(value=1.0, continuous_update=False, description='m1', max=2.0, min=-2.0, step=0.05), FloatSlider(value=2.0, continuous_update=False, description='m2', max=2.0, min=-2.0, step=0.05), FloatSlider(value=0.2, continuous_update=False, description='m1_center', max=2.0, min=-2.0, step=0.05), FloatSlider(value=0.2, continuous_update=False, description='dm1', max=0.5, step=0.05), FloatSlider(value=0.75, continuous_update=False, description='m2_center', max=2.0, min=-2.0, step=0.05), FloatSlider(value=0.07, continuous_update=False, description='sigma_2', max=0.1, min=0.01, step=0.01), SelectMultiple(description='option', index=(1,), options=('kernel', 'model', 'data'), value=('model',)), Checkbox(value=False, description='add_noise'), FloatText(value=20.0, description='percentage'), FloatText(value=0.001, description='floor'), Output()), _dom_classes=('widget-interact',))

## Inverse Problem

In the inverse problem we attempt to find the model $\mathbf{m}$ that gave rise to the observational data $\mathbf{d}^{obs}$. The inverse problem is formulated as an optimization problem: 


$$\text{minimize} \ \ \ \phi(\mathbf{m}) = \phi_d(\mathbf{m}) + \beta \phi_m(\mathbf{m}) $$

where 

- $\phi_d$: data misfit
- $\phi_m$: model regularization
- $\beta$: trade-off (or Tikhonov) parameter  $0<\beta<\infty$

Data misfit is defined as 

$$ \phi_d = \Sigma_{j=1}^{N}\Big(\frac{\mathbf{g}_j\mathbf{m}-d^{obs}_j}{\epsilon_j}\Big)^2$$

where $\epsilon_j$  is an estimate of the standard deviation of the $j$th datum.





The model regularization term, $\phi_m$, can be written as 

$$ \phi_m(\mathbf{m}) = \alpha_s \int (\mathbf{m}-\mathbf{m}_{ref}) dx + \alpha_x \int (\frac{d \mathbf{m}}{dx}) dx$$

The first term is referred to as the "smallness" term. Minimizing this generates a model that is close to a reference model $m_{ref}$. The second term penalizes roughness of the model. It is generically referred to as a "flattest" or "smoothness" term.  

## Step3: Invert the data, and explore inversion results



In the inverse problem we define parameters needed to evaluate the data misfit and the model regularization terms. We then deal with parameters associated with the inverion 

**comments**
Data misfit: percentage, floor and chifact
Model object fcn: alphas, mref (at the end)
Inversion: mref, add 

In [8]:
Q3 = app.interact_plot_inversion()

interactive(children=(IntText(value=20, description='maxIter'), FloatSlider(value=0.0, continuous_update=False, description='m0', max=2.0, min=-2.0, step=0.05), FloatSlider(value=0.0, continuous_update=False, description='mref', max=2.0, min=-2.0, step=0.05), FloatText(value=20.0, description='percentage'), FloatText(value=0.001, description='floor'), FloatText(value=1.0, description='chifact'), FloatText(value=100.0, description='beta0_ratio'), FloatSlider(value=2.0, continuous_update=False, description='coolingFactor', max=10.0, min=0.1, step=1.0), IntSlider(value=1, continuous_update=False, description='coolingRate', max=10, min=1), FloatText(value=1.0, description='alpha_s'), FloatText(value=0.0, description='alpha_x'), Checkbox(value=False, description='use_target'), Checkbox(value=True, description='run'), ToggleButtons(description='option', options=('misfit', 'tikhonov'), value='misfit'), IntSlider(value=0, continuous_update=False, description='i_iteration', max=20), Output()), _dom_classes=('widget-interact',))